In [17]:
import ee
import geopandas as gpd
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Initialize the Earth Engine API.
ee.Authenticate()
ee.Initialize()

# -----------------------------
# 1. Load your AOI from a shapefile
# -----------------------------
shp_path = r"C:\Users\konst\Documents\Hiwi\mw3\drought_indicies\data\Untersuchungsgebiete\002a_Sen1-Subset_Eifel.shp"
gdf = gpd.read_file(shp_path)
geojson = gdf.to_json()
aoi = ee.Geometry(json.loads(geojson)['features'][0]['geometry'])

# -----------------------------
# 2. Define date range and cloud mask function
# -----------------------------
start_date = '2000-01-01'
end_date   = '2025-01-31'

# -----------------------------
# 3. Load and filter the two HLS collections (Sentinel and Landsat)
# -----------------------------
landsat7 = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
         )


# Rename bands for consistency
landsat7_renamed = landsat7.map(lambda img: img.select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA_PIXEL']
))

# Define the cloud mask function
def mask_clouds(image):
    # Bits 0 (Fill), 1 (Dilated Cloud), 3 (Cloud), 4 (Cloud Shadow)
    cloud_shadow_bitmask = (1 << 0) | (1 << 1) | (1 << 3) | (1 << 4)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0)
    return image.updateMask(mask)

# Apply the cloud mask to the Landsat 7 collection
landsat7_masked = landsat7_renamed.map(mask_clouds)